In [1]:
import sys
sys.path.append("..")
import json
import urllib.request
from IPython.display import clear_output
import sources
from countries.list import CountryList
from data.Frame import Frame
from data.Stats import calculate_weighted_average as wa
c = CountryList()
c.load_wb()
from sources.worldbank.indicators import get_data_frame_wide as getdfw

In [2]:
def get_data_table(start, end, indicator, groups, export_to_excel=False):    
    # We will store the final result here
    ndf = Frame()
    # Now we get the data that we want to average by region (defined in the groups)
    # and also total
    for g in groups:    
        # Data points
        df_data = getdfw(name=indicator, 
            start=start, end=end, 
            countries=c.get_groups(g),
            label_field='entity',
            value_field='value',
            column_field='date'
            )

        for r in df_data.rows:
            ndf.rows.append(r)
    
    # Produce dataframe that will be resturned as a result of the function
    final_df = Frame()    
    final_df.add_column(ndf.get_column('entity'))
    
    for i in range(end-start):
        final_df.add_column(ndf.get_column(float('%s%s.0' % (str(start)[0:4-len(str(i))],i))))
            
    if export_to_excel == True:
        s = ""
        for g in groups:    
            for i in g:
                s =  i + "_" + s     
        final_df.name = "%s-%s-%s-%s" % (indicator, start, end, s)
        final_df.to_xlsx()
    
    return final_df


In [7]:
# Added value of agriculture as a percentage of GDP
start = 2000
end = 2018
indicator = 'NV.AGR.TOTL.ZS'
weight = 'NY.GDP.MKTP.CD'
groups_ldcs=[['LDCs', 'Africa'],['LDCs', 'Asia and the Pacific'],['LDCs', 'America']]
groups=[['Developing excluding LDCs']]

In [8]:
data_ldcs = get_data_table(start, end, indicator, groups_ldcs, export_to_excel=False)
weight_ldcs = get_data_table(start, end, weight, groups_ldcs, export_to_excel=False)

In [9]:
df_ldcs_w = wa(data_ldcs, weight_ldcs, 'entity' )

In [10]:
df_ldcs_w.to_xlsx()

In [12]:
# Employment in agriculture as a percentage of total employment
start = 2000
end = 2018
indicator = 'SL.AGR.EMPL.ZS'
weight = 'SP.RUR.TOTL'
groups_ldcs=[['LDCs', 'Africa'],['LDCs', 'Asia and the Pacific'],['LDCs', 'America']]
groups=[['Developing excluding LDCs']]

data_ldcs = get_data_table(start, end, indicator, groups_ldcs, export_to_excel=False)
weight_ldcs = get_data_table(start, end, weight, groups_ldcs, export_to_excel=False)
df_ldcs_w = wa(data_ldcs, weight_ldcs, 'entity' )
df_ldcs_w.to_xlsx()